In [2]:
import sqlite3
import pandas as pd
import re

In [3]:
conn = sqlite3.connect('Powerball_SQLDB.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Joined_Table')
conn.commit()
results = cursor.fetchall()
for row in results:
    data = []
    for row in results:
        data.append(row[:47])
    df = pd.DataFrame(data)

conn.close()

In [4]:
df.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,181,2015-11-04,$144 Million,Avondale Revocable Trust,Tennessee,8,2015-11-04,2,12,17,20,65,17,4.0
1,182,2016-01-13,$1.58 Billion,Marvin and Mae Acosta,California,28,2016-01-13,4,8,19,27,34,10,2.0
2,185,2016-03-02,$291.4 Million,James Stocklas,Florida,42,2016-03-02,12,13,44,52,62,6,2.0
3,186,2016-05-07,$284 Million,The Smith Family Lottery Pool,New Jersey,61,2016-05-07,5,25,26,44,66,9,2.0
4,187,2016-07-30,$487 Million,Robin Egg 2016 Nominee Trust,New Hampshire,85,2016-07-30,11,17,21,23,32,5,2.0
5,188,2016-09-17,$246 Million,William and Heather ten Broeke,Georgia,99,2016-09-17,9,19,51,55,62,14,4.0
6,189,2016-11-26,$420.9 Million,Tennessee 20,Tennessee,119,2016-11-26,17,19,21,37,44,16,2.0
7,190,2016-12-17,$121.6 Million,Not Disclosed,DE,125,2016-12-17,1,8,16,40,48,10,2.0


In [5]:
df.columns = ['Index', 'Draw_date', 'Jackpot_Prize', 'Winners', 'State', 'Index_2', "Draw_Date2", "WB1", "WB2", 'WB3', 'WB4', 'WB5', 'PB', 'Multiplier']
df.head(8)

,Index,Draw_date,Jackpot_Prize,Winners,State,Index_2,Draw_Date2,WB1,WB2,WB3,WB4,WB5,PB,Multiplier
0,181,2015-11-04,$144 Million,Avondale Revocable Trust,Tennessee,8,2015-11-04,2,12,17,20,65,17,4.0
1,182,2016-01-13,$1.58 Billion,Marvin and Mae Acosta,California,28,2016-01-13,4,8,19,27,34,10,2.0
2,185,2016-03-02,$291.4 Million,James Stocklas,Florida,42,2016-03-02,12,13,44,52,62,6,2.0
3,186,2016-05-07,$284 Million,The Smith Family Lottery Pool,New Jersey,61,2016-05-07,5,25,26,44,66,9,2.0
4,187,2016-07-30,$487 Million,Robin Egg 2016 Nominee Trust,New Hampshire,85,2016-07-30,11,17,21,23,32,5,2.0
5,188,2016-09-17,$246 Million,William and Heather ten Broeke,Georgia,99,2016-09-17,9,19,51,55,62,14,4.0
6,189,2016-11-26,$420.9 Million,Tennessee 20,Tennessee,119,2016-11-26,17,19,21,37,44,16,2.0
7,190,2016-12-17,$121.6 Million,Not Disclosed,DE,125,2016-12-17,1,8,16,40,48,10,2.0


## I dont want to drop these one by one, but this DF is weird

In [6]:
df = df.drop('Index', axis=1)

In [7]:
df = df.drop('Index_2', axis=1)

In [8]:
df = df.drop('Winners', axis=1)

In [9]:
df = df.drop('Draw_Date2', axis=1)

In [10]:
df = df.drop('Multiplier', axis=1)

In [11]:
df.value_counts().head(8)

Draw_date   Jackpot_Prize   State       WB1  WB2  WB3  WB4  WB5  PB
2015-11-04  $144 Million    Tennessee   2    12   17   20   65   17    1
2020-09-16  $94 Million     New York    10   17   31   51   53   1     1
2019-09-04  $80 Million     New York    4    8    30   52   59   2     1
2019-09-21  $80 Million     Michigan    1    9    22   36   68   22    1
2019-11-02  $150 Million    California  3    23   32   37   58   22    1
2020-01-29  $396.9 Million  Florida     9    12   15   31   60   2     1
2020-02-15  $70 Million     Michigan    16   32   35   36   46   3     1
2020-04-08  $190 Million    New Jersey  2    37   39   48   54   5     1
Name: count, dtype: int64

In [12]:
merged = pd.concat([df['WB1'], df['WB2'], df['WB3'], df['WB4'], df['WB5']])
merged.mode()

0    52
dtype: object

In [13]:
merged.value_counts().head(8)

52    8
9     7
17    7
12    6
8     6
44    6
6     6
27    6
Name: count, dtype: int64

In [14]:
numbers_frequency = df['PB'].value_counts()
top_5_numbers_frequency = numbers_frequency.head(5)
print(top_5_numbers_frequency)

PB
17    4
4     4
24    4
22    3
14    3
Name: count, dtype: int64


In [15]:
numbers_frequency = df['State'].value_counts()
top_5_numbers_frequency = numbers_frequency.head(5)
print(top_5_numbers_frequency)

State
New York      6
Florida       5
New Jersey    5
Wisconsin     4
Tennessee     3
Name: count, dtype: int64


In [16]:
def convert_to_numeric(x):
    return int(re.findall(r'\d+', x)[0])

money = df['Jackpot_Prize'].map(convert_to_numeric)
df['Jackpot_Prize'] = money

In [17]:
df.head(8)

,Draw_date,Jackpot_Prize,State,WB1,WB2,WB3,WB4,WB5,PB
0,2015-11-04,144,Tennessee,2,12,17,20,65,17
1,2016-01-13,1,California,4,8,19,27,34,10
2,2016-03-02,291,Florida,12,13,44,52,62,6
3,2016-05-07,284,New Jersey,5,25,26,44,66,9
4,2016-07-30,487,New Hampshire,11,17,21,23,32,5
5,2016-09-17,246,Georgia,9,19,51,55,62,14
6,2016-11-26,420,Tennessee,17,19,21,37,44,16
7,2016-12-17,121,DE,1,8,16,40,48,10


In [18]:
df = df.rename(columns={'Jackpot_Prize': 'Jackpot_Prize(Millions)'})

In [19]:
df.loc[1, 'Jackpot_Prize(Millions)'] = 1580 #Modified to 1000 from 1, because it's a billion

In [20]:
#The lump sum amount after Taxes of approx. 52%  In Millions
lump_sum = df['Jackpot_Prize(Millions)'] * .52
lump_sum.head(8)

0     74.88
1    821.60
2    151.32
3    147.68
4    253.24
5    127.92
6    218.40
7     62.92
Name: Jackpot_Prize(Millions), dtype: float64

In [21]:
#Amount paid in Taxes in Millions
lump_sum_tax_amount = df['Jackpot_Prize(Millions)'] * .48
lump_sum_tax_amount.head(8)

0     69.12
1    758.40
2    139.68
3    136.32
4    233.76
5    118.08
6    201.60
7     58.08
Name: Jackpot_Prize(Millions), dtype: float64

In [22]:
#The average the Winner would recieve each year, over 30 years.  After taxes.  In Millions.
annuity = df['Jackpot_Prize(Millions)'] / 30
annuity = annuity * .63
annuity.head(8)

0     3.024
1    33.180
2     6.111
3     5.964
4    10.227
5     5.166
6     8.820
7     2.541
Name: Jackpot_Prize(Millions), dtype: float64

In [23]:
# Amount paid every year for winner annuity in Taxes, in Millions 
annuity_taxes_paid = df['Jackpot_Prize(Millions)'] / 30
annuity_taxes_paid = annuity_taxes_paid * .37
annuity_taxes_paid.head(8)

0     1.776000
1    19.486667
2     3.589000
3     3.502667
4     6.006333
5     3.034000
6     5.180000
7     1.492333
Name: Jackpot_Prize(Millions), dtype: float64

In [24]:
#df.to_csv('SQLite_Table_DB.csv')